In [1]:
import pandas as pd
import smtplib
import os
import json
import matplotlib.pyplot as plt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders

In [2]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'CREDENCIALES DE MINTICLOUD'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = False
ENVIO_A_PENDIENTES = False

if CORREO_TEST:
    RUTA = './INPUT/MINTICLOUD/TEST.xlsx'
elif ENVIO_A_PENDIENTES:
    RUTA = './INPUT/MINTICLOUD/CREDENCIALES_EMAIL_PENDIENTES.xlsx'
else:
    RUTA = './INPUT/MINTICLOUD/CREDENCIALES.xlsx'

COLUMNA_CORREO = 'CORREO'
COLUMNAS = [
    'CURSO',
    'USUARIO',
    'PASS',
]

In [3]:
MSG_HEAD = """<html><body><p>Bucaramanga, 5 de septiembre<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022 - Ciclo 3</p>

"""
MSG_BODY = """<p>Atendiendo el asunto, adjunto a este correo se encuentran las credenciales de MintiCloud de el/los cursos que usted tiene asignados<br>
El archivo adjunto contiene 3 columnas:</p>
<ul>
<li><b>Curso:</b> El curso al que le corresponden las credenciales</li>
<li><b>Usuario:</b> El nombre de usuario tanto para el servidor FTP como para la base de datos</li>
<li><b>Pass:</b> La contraseña tanto para el servidor FTP como para la base de datos</li>
</ul>
<p>Tenga en cuenta que la URL principal del hosting compartido brindado es <a href="https://minticloud.uis.edu.co/">https://minticloud.uis.edu.co</a>. <br>
Para conectarse a través de un software de gestión FTP (se les recomienda Filezilla), <b>es necesario usar el protocolo sftp</b> (sftp://minticloud.uis.edu.co/) <br>
Recuerden que tienen una cuenta especial para ustedes por cada curso y que es una credencial por cada grupo (grupo1, grupo2, ...).</p>
"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [4]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [5]:
datos

,CURSO,RESPONSABLE,CORREO,USUARIO,PASS,PATH,DB
0,O1,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,c3s1formador,3uLeULVV,/var/sftp/webapps/c3s1formador,c3s1formador
1,O1,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,c3s1grupo1,TtI2dOpq,/var/sftp/webapps/c3s1grupo1,c3s1grupo1
2,O1,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,c3s1grupo2,q6hTMtDJ,/var/sftp/webapps/c3s1grupo2,c3s1grupo2
3,O1,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,c3s1grupo3,RkSX9Pu1,/var/sftp/webapps/c3s1grupo3,c3s1grupo3
4,O1,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,c3s1grupo4,BFecgS3W,/var/sftp/webapps/c3s1grupo4,c3s1grupo4
...,...,...,...,...,...,...,...
1134,Tutor,KEVIN ALEXANDER RINCON ANGEL,rinconkevin011@gmail.com,c3tutor10,zOG2vf2S,/var/sftp/webapps/c3tutor10,c3tutor10
1135,Tutor,MIRNA LORENA GONZALEZ SANTAMARIA,mirnagonzalez1721@gmail.com,c3tutor11,vzEA2m1w,/var/sftp/webapps/c3tutor11,c3tutor11
1136,Tutor,JESUS LEONARDO PEREZ CASTRO,jlperezca@ufpso.edu.co; leonardoperez1204@gmai...,c3tutor12,Lqdg95MK,/var/sftp/webapps/c3tutor12,c3tutor12
1137,Tutor,KEVIN ALONSO LUNA BUSTOS,kevinluna00@gmail.com,c3tutor13,8guDWcAJ,/var/sftp/webapps/c3tutor13,c3tutor13


In [6]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

def getCuerpoCorreo(multipart, formador, df): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = ''
    # Contenido adicional que se quiera añadir
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo

def enviarCorreo(origen, destino, asunto, mensaje, adjuntos = []):
    multipart = MIMEMultipart()
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart.attach(MIMEText(mensaje))
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [7]:
enviados = datos.copy()
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['RESPONSABLE'].iloc[0]
    adjuntos = []
    nombreArchivo = 'MINTICLOUD_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido = getCuerpoCorreo(base, nombreFormador, dfD)
    base = getAdjuntos(base, [], adjuntos)
    try:        
        servidor.sendmail(ORIGEN, destino, base.as_string())
        enviados = enviados[enviados[COLUMNA_CORREO] != destino]
    except:
        enviados.to_excel('./INPUT/MINTICLOUD/CREDENCIALES_EMAIL_PENDIENTES.xlsx', index=False)
        print ('Ocurrió un error al enviar el correo a', destino)
        break
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()

1 Enviado a DARIO ALEJANDRO RIANO VELANDIA alejandroria.94@gmail.com
2 Enviado a OSCAR MAURICIO GONZALEZ GOMEZ om5@misena.edu.co
3 Enviado a ROGERIO ORLANDO BELTRAN CASTRO rogeriobeltran@gmail.com
4 Enviado a CARLOS ANDRES PALMA SUAREZ carlospalma_sistemas@hotmail.com
5 Enviado a YHARY ESTEFANIA ARIAS TRILLOS yharystefa@gmail.com
6 Enviado a SILVIA JULIANA  MORENO ROA julianamroa@gmail.com
7 Enviado a SERGIO ARTURO MEDINA CASTILLO smedina_castillo@yahoo.es
8 Enviado a CARLOS HORACIO GARCIA CASTRILLON carloshgcastrillon@gmail.com
9 Enviado a CARLOS ARTURO PARRA ORTEGA carapa2000@hotmail.com
10 Enviado a EDWARD ALBERTO ROPERO PEREZ edward_a_ropero@hotmail.com
11 Enviado a SERGIO NICOLAS PINILLA CANON pinilla.nicolas10@gmail.com
12 Enviado a YESID OSWALDO QUINTERO MARTINEZ yesidquintero06@gmail.com
13 Enviado a JHON JAIRO CORTES PAREDES jhonjairo.cortesp@gmail.com
14 Enviado a JAHIR AUGUSTO SAAVEDRA GARCIA jahir.saavedra@gmail.com
15 Enviado a EDILBERTO SIERRA MEJIA ingeniero.beto@hotmail

(221, b'2.0.0 Service closing transmission channel')